In [1]:
import findspark
findspark.init()
from pyspark import SparkContext
from pyspark.sql.session import SparkSession
sc = SparkContext("local", "first app")
spark = SparkSession(sc)
import plotly.graph_objects as go
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [2]:
text_file = sqlContext.read.format('com.databricks.spark.csv').options(header='true', inferschema='true', quote='"', delimiter=',').load("C:\\Users\\sharolin\\Desktop\\entrada\\TraficoAereoGt.csv")
rddfiltro = text_file.rdd.map(tuple)
rddAERO = rddfiltro.map(lambda word: (word[0].split('/')[1],word[5]))
rddAERO.take(3)

[('01', 195), ('02', 112), ('03', 159)]

In [3]:
rddCONTEO=rddAERO.reduceByKey(lambda a,b: a+b)
print("Conteo total -> %s" % rddCONTEO.collect())

Conteo total -> [('01', 6034), ('02', 5912), ('03', 5600), ('04', 5723), ('05', 5690), ('06', 5476), ('07', 5972), ('08', 5850), ('09', 5425), ('10', 5347), ('11', 6467), ('12', 5862)]


In [5]:
rddORDEN=sc.parallelize(rddCONTEO.sortBy(lambda a: a[1],False).take(3))
print("meses con mayor numero de pasajeros de salida  -> %s" % rddORDEN.collect())

meses con mayor numero de pasajeros de salida  -> [('11', 6467), ('01', 6034), ('07', 5972)]


In [7]:
rddNombres = rddORDEN.map(lambda x: (x[0]))
rddTotales = rddORDEN.map(lambda x: (x[1]))
print(rddNombres.collect())
print(rddTotales.collect())

['11', '01', '07']
[6467, 6034, 5972]


In [8]:
fig = go.Figure(data=go.Pie(labels=rddNombres.collect(),values=rddTotales.collect()))
fig.update_layout(title_text='Meses con mayor numero de pasajeros de salida',title_font_size=30)
fig.update_traces(hoverinfo='label+percent', textinfo='value',textfont_size=28)
fig.write_html('R2_A2.html', auto_open=True)